In [14]:
import openai
import json
import math
from openai.lib._parsing._completions import type_to_response_format_param
import pandas as pd

import functions.prompts as prompts

client = openai.Client()

In [ ]:
df = pd.read_csv("dump/csv/result_a.csv")
df.head()

,id,file,name,journal,authors,affiliations,len-original,len-anond,anthropic-score-1,anthropic-score-2,...,anthropic-rigor-2,anthropic-scope-2,anthropic-impact-2,anthropic-written_by_ai-2,anthropic-originality-3,anthropic-rigor-3,anthropic-scope-3,anthropic-impact-3,anthropic-written_by_ai-3,rank
0,1_0,1. Econometrica/ecta200736.pdf,The Political Economy of Zero-Sum Thinking,Econometrica,S. Nageeb Ali; Maximilian Mihm; Lucas Siga,"Department of Economics, Pennsylvania State Un...",16496.0,15956.0,4,4,...,9,9,10,1,8,9,9,9,1,1
1,1_1,1. Econometrica/ecta200731.pdf,Social Media and Collective Action in China,Econometrica,Bei Qin; David Strömberg; Yanhui Wu,"Bei Qin: Department of Accountancy, Economics ...",18206.0,17419.0,6,6,...,10,9,10,1,9,10,9,10,1,1
2,1_2,1. Econometrica/ecta200725.pdf,Ambiguous Contracts,Econometrica,Paul Dütting; Michal Feldman; Daniel Peretz; L...,"Google Research; School of Computer Science, T...",11830.0,11448.0,4,4,...,9,9,8,1,8,9,9,8,1,1
3,1_3,1. Econometrica/ecta200741.pdf,PERSUASION MEETS DELEGATION,Econometrica,Anton Kolotilin; Andriy Zapechelnyuk,"Anton Kolotilin: School of Economics, UNSW Bus...",15756.0,15075.0,4,6,...,9,9,8,1,9,10,10,9,1,1
4,1_4,1. Econometrica/Econometrica - 2025 - Berger -...,"Minimum Wages, Efficiency, and Welfare",Econometrica,David Berger; Kyle Herkenhoff; Simon Mongey,"Economics Department, Duke University; Departm...",18607.0,16735.0,4,5,...,10,9,10,0,8,9,9,10,1,1


In [ ]:
def wrap(body, id):
    return {
        "custom_id": id,
        "method": "POST",
        "url": "/v1/chat/completions",
        "body": body
    }
    
def gen_body(text, top5=True):
    return {
            "model": "gpt-4o-mini",
            "messages":[{
                    "role": "system",
                    "content": prompts.top5() if top5 else prompts.analysis()
                },
                {
                    "role": "user",
                    "content": text
                }],
            "response_format": type_to_response_format_param(prompts.Top5Model if top5 else prompts.AnalysisModel)
        }


In [ ]:
import numpy as np

divider = 5

for index, row in df.iterrows():
    with open(f'output/{row["id"]}.txt', 'r') as f:
        text = f.read()
        paper = f"PAPER TITLE: {row['name']}\n\nPAPER TEXT: {text}"
        # [ wrap(gen_body(f"PAPER TITLE: {row['name']}\n\nPAPER TEXT: {text}"), f"{row["id"]}_{i}") for i in range(3) ]
        id = [  *[ wrap( gen_body(paper),                f"{row["id"]}|{i}+top5") for i in range(3) ],
                *[ wrap( gen_body(paper, top5=False),    f"{row['id']}|{i}+analysis") for i in range(3) ] ]
        
        file_index = math.floor(index / (len(df) / divider))
        # file_index = 0

        with open(f"input/batch-{file_index}.jsonl", 'a') as f:
            f.write(f"{"\n".join([json.dumps(i) for i in id])}\n")

In [ ]:
batch_input_files = []

for i in range(divider):
    print(f"Sending batch {i}")
    batch_input_file = client.files.create(
        file=open(f"input/batch-{i}.jsonl", "rb"),
        purpose="batch"
    )
    print(f"{i} {batch_input_file.id}")
    batch_input_files.append(batch_input_file)

print(batch_input_files)

In [ ]:
# KINDA DANGEROUS
# batch_objects = []
# for i, b in enumerate(batch_input_files):
#     batch_input_file_id = b.id
#     batch_object = client.batches.create(
#         input_file_id=batch_input_file_id,
#         endpoint="/v1/chat/completions",
#         completion_window="24h",
#         metadata={
#             "description": f"[AI REVIEWER] Set {i}"
#         }
#     )
#     print(f"{i} {batch_object.id}")
#     batch_objects.append(batch_object)

In [18]:
batches = []
with open("dump/openai-batches.txt", 'r') as f:
    batches = f.read()
    batches = batches.split("\n")

In [24]:
file_ids = []
for i, batch in enumerate(batches):
    x = client.batches.retrieve(batch)
    file_ids.append(x.output_file_id)

In [25]:
file_ids

['file-MK6mgmciFdGAPbRmbAhR61',
 'file-RjUzYiHfmJpZtJbmaMjH5j',
 'file-JFJPrdj9N44wbNyCG7PCGm',
 'file-B5EqjTpoLdbPmJZCjWZzt1',
 'file-ARzbX5NuinoJpw7VJkBjxc']

In [27]:
for i, id in enumerate(file_ids):
    file_response = client.files.content(id)
    with open(f"dump/eval-output/openai-output.jsonl", 'a') as f:
        f.write(file_response.text)

In [31]:
import pandas as pd
import json

df = pd.read_csv("dump/csv/result_a.csv")

f = open("dump/eval-output/openai-output.jsonl", "r")
file_response = f.read()
f.close()

for line in file_response.split("\n")[:-1]:
    l = json.loads(line)
    id = l['custom_id'].split("|")[0]
    no, typ = l['custom_id'].split("|")[1].split("+")
    
    idx = df.index[df['id'] == id].tolist()[0]
    
    content = l['response']['body']['choices'][0]['message']['content']
    
    metrics = ['score'] if typ == "top5" else ['originality', 'rigor', 'scope', 'impact', 'written_by_ai']
    validateModel = prompts.Top5Model if typ == "top5" else prompts.AnalysisModel

    for i, metric in enumerate(metrics):
        column_name = f"openai-{metric}-{int(no)+1}"
        
        if column_name not in df.columns:
            df[column_name] = None

        o = validateModel.model_validate_json(content)
        df.loc[idx, column_name] = o.__dict__[metric]

In [32]:
df

,id,file,name,journal,authors,affiliations,len-original,len-anond,anthropic-score-1,anthropic-score-2,...,openai-originality-2,openai-rigor-2,openai-scope-2,openai-impact-2,openai-written_by_ai-2,openai-originality-3,openai-rigor-3,openai-scope-3,openai-impact-3,openai-written_by_ai-3
0,1_0,1. Econometrica/ecta200736.pdf,The Political Economy of Zero-Sum Thinking,Econometrica,S. Nageeb Ali; Maximilian Mihm; Lucas Siga,"Department of Economics, Pennsylvania State Un...",16496.0,15956.0,4,4,...,8,7,7,9,2,8,9,8,7,0
1,1_1,1. Econometrica/ecta200731.pdf,Social Media and Collective Action in China,Econometrica,Bei Qin; David Strömberg; Yanhui Wu,"Bei Qin: Department of Accountancy, Economics ...",18206.0,17419.0,6,6,...,8,9,8,9,1,8,9,8,9,1
2,1_2,1. Econometrica/ecta200725.pdf,Ambiguous Contracts,Econometrica,Paul Dütting; Michal Feldman; Daniel Peretz; L...,"Google Research; School of Computer Science, T...",11830.0,11448.0,4,4,...,8,9,7,8,1,8,9,7,8,2
3,1_3,1. Econometrica/ecta200741.pdf,PERSUASION MEETS DELEGATION,Econometrica,Anton Kolotilin; Andriy Zapechelnyuk,"Anton Kolotilin: School of Economics, UNSW Bus...",15756.0,15075.0,4,6,...,9,8,7,8,1,9,8,7,6,0
4,1_4,1. Econometrica/Econometrica - 2025 - Berger -...,"Minimum Wages, Efficiency, and Welfare",Econometrica,David Berger; Kyle Herkenhoff; Simon Mongey,"Economics Department, Duke University; Departm...",18607.0,16735.0,4,5,...,8,9,8,7,1,8,9,7,8,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1095,P10_5,P10. Business and Econ Journal/the-impact-of-a...,The Impact of Adoption of Agricultural Product...,Business and Econ Journal,Getaye Gizaw,"Getaye Gizaw, Department of Principal investig...",2951.0,2807.0,2,3,...,6,8,7,7,1,7,8,6,8,2
1096,P10_6,P10. Business and Econ Journal/effect-of-workl...,Effect of Work-Life Balance on Job Satisfactio...,Business and Econ Journal,Fartun Ahmed Sheikh Mohamud,"Fartun Ahmed Sheikh Mohamud, Department of Man...",4284.0,4046.0,2,2,...,7,8,6,7,1,6,7,6,7,0
1097,P10_7,P10. Business and Econ Journal/an-exploratory-...,An Exploratory Study on the Viability of Corpo...,Business and Econ Journal,Dr. Mesay Ayele,"Department of Management, Hawassa University, ...",9101.0,8039.0,3,2,...,8,6,7,7,0,7,6,5,8,2
1098,P10_8,P10. Business and Econ Journal/centre-peripher...,Centre Periphery Relationship: An Analysis Abo...,Business and Econ Journal,Jorge Italo de Lima,"Jorge Italo de Lima, Department of Textile and...",7311.0,7117.0,3,3,...,7,8,7,8,0,7,8,6,7,0


In [33]:
df.to_csv("result_oa.csv", index=False)